In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=5da57b3025e8bf89f6eace5ea0d295e133b2e90ed41d9d10402410ecff54f3fd
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=991bea38c40c8fcc96d56ff519a240d38f3b32d3534ce773fc84a184aa190e3a
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)

2.3.0


In [3]:
fashion = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images.shape

(60000, 28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)   
train_images.shape

(60000, 28, 28, 1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output')

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4054 - accuracy: 0.8544 - val_loss: 0.3029 - val_accuracy: 0.8897
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2708 - accuracy: 0.9002 - val_loss: 0.2681 - val_accuracy: 0.8997
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2155 - accuracy: 0.9205 - val_loss: 0.2528 - val_accuracy: 0.9092


Epoch 1/3
1688/1688 [==============================] - 10s 6ms/step - loss: 0.5632 - accuracy: 0.8281 - val_loss: 0.3798 - val_accuracy: 0.8597
Epoch 2/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.3672 - accuracy: 0.8652 - val_loss: 0.3519 - val_accuracy: 0.8660
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3294 - accuracy: 0.8797 - val_loss: 0.3650 - val_accuracy: 0.8608


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4800 - accuracy: 0.8271 - val_loss: 0.4083 - val_accuracy: 0.8467
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3641 - accuracy: 0.8666 - val_loss: 0.3522 - val_accuracy: 0.8702
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3256 - accuracy: 0.8788 - val_loss: 0.3849 - val_accuracy: 0.8618


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4124 - accuracy: 0.8515 - val_loss: 0.2958 - val_accuracy: 0.8892
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2694 - accuracy: 0.9012 - val_loss: 0.2940 - val_accuracy: 0.8928
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2153 - accuracy: 0.9192 - val_loss: 0.2528 - val_accuracy: 0.9095


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4141 - accuracy: 0.8518 - val_loss: 0.3047 - val_accuracy: 0.8907
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2661 - accuracy: 0.9036 - val_loss: 0.2571 - val_accuracy: 0.9048
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2141 - accuracy: 0.9208 - val_loss: 0.2520 - val_accuracy: 0.9097


INFO:tensorflow:Oracle triggered exit


In [13]:
model = tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                743456    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 821,274
Trainable params: 821,274
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1707 - accuracy: 0.9366 - val_loss: 0.2714 - val_accuracy: 0.9080
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1380 - accuracy: 0.9489 - val_loss: 0.2744 - val_accuracy: 0.9100
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1068 - accuracy: 0.9599 - val_loss: 0.2821 - val_accuracy: 0.9128
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0843 - accuracy: 0.9684 - val_loss: 0.3220 - val_accuracy: 0.9122
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0652 - accuracy: 0.9757 - val_loss: 0.3457 - val_accuracy: 0.9160
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0518 - accuracy: 0.9806 - val_loss: 0.4011 - val_accuracy: 0.9145
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0417 - accuracy: 0.9849 - val_loss: 0.4292 - val_accuracy

In [27]:
test_images = test_images.reshape(len(test_images),28,28,1)
print(test_images.shape)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

(10000, 28, 28, 1)
313/313 [==============================] - 1s 3ms/step - loss: 0.4564 - accuracy: 0.9102
Test accuracy: 0.9101999998092651
